# Filtering

Filter on:
1. Invalid messages
2. outside timeframe
3. non english tweets
4. duplicated tweets/spam users
5. retweets


Postgres -> Pandas:
> dataframe = psql.read_sql("SELECT * FROM ", connection)

In [1]:
import os
from dotenv import find_dotenv, load_dotenv
import psycopg2 as pg
import pandas.io.sql as psql

In [2]:
# Load variables from .env, connect to DB
load_dotenv(find_dotenv())
database_url = os.environ.get('DATABASE_URL')

conn = pg.connect(database_url)
curr = conn.cursor()

Count how many tweets we have total.

In [3]:
curr.execute('SELECT COUNT(*) FROM raw_tweets;')
num_total_tweets = curr.fetchone()[0]
print ('Total tweets:\t{}'.format(num_total_tweets))

Total tweets:	617512


### Valid Messages

Get count of both NULL and NOT NULL in one query:
> select count(*) - count(a), count(a) from us;

In [4]:
curr.execute('SELECT count("message"), COUNT(*) - COUNT("message") from raw_tweets;')
num_valid_message, num_null_message = curr.fetchone()

print ('Valid:\t{}\nInvalid\t{}'.format(num_valid_message, num_null_message))

Valid:	616039
Invalid	1473


### Timestamps and Dates

- Date of: Jan 27th, 2017
- Min Date (-4 weeks): Friday, December 30, 2016
- Max Date (+4 weeks): Friday, February 24, 2017


> WHERE datefield >= '2010-01-01 00:00:00' 

In [5]:
date_range_query = "SELECT MIN(date), MAX(date) FROM raw_tweets;"
curr.execute(date_range_query)
date_min, date_max = curr.fetchone()

print ('Min Date:\t{}\nMax Date:\t{}'.format(date_min, date_max))

date_filter_query = "SELECT COUNT(*) FROM raw_tweets WHERE date >= '2016-12-30 04:00:00' AND date < '2017-02-24 04:00:00';"
curr.execute(date_filter_query)
num_valid_date = curr.fetchone()[0]
print ('Valid date range tweets:\t{}'.format(num_valid_date))

Min Date:	2016-12-22 17:36:32
Max Date:	2018-03-26 03:59:56
Valid date range tweets:	0


### Language

In [6]:
filter_language = "SELECT COUNT(*) FROM raw_tweets WHERE LEFT(language, 2) LIKE 'en';"
curr.execute(filter_language)
num_english_tweets = curr.fetchone()[0]

print ('English tweets:\t{}'.format(num_english_tweets))
print ('Non english tweets:\t{}'.format(num_total_tweets-num_english_tweets))

English tweets:	287783
Non english tweets:	329729


### Duplicated Tweets and Spammy users

> select a,b from dup group by a,b having count(*) > 1; 
> 
> select regno from mytable group by regno having count(stockno) > 1;
>
> SELECT [EmailAddress], [CustomerName] FROM [Customers] WHERE [EmailAddress] IN
  (SELECT [EmailAddress] FROM [Customers] GROUP BY [EmailAddress] HAVING COUNT(*) > 1)

In [7]:
dupe_query = """
SELECT 
    "username", "tweetID", "message" 
FROM 
    raw_tweets 
WHERE 
    "tweetID" IN (SELECT "tweetID" FROM raw_tweets GROUP BY "tweetID" HAVING COUNT(*) > 1);
"""


dupe_df = psql.read_sql(dupe_query, conn)
dupe_df.head(20)

,username,tweetID,message
0,jacko1luv,978013654363918336,RT : Turkish leaders; including Muslim Preside...
1,Hieiandshino,978019727653199872,RT : Just like they’ve smeared protesters duri...
2,Hieiandshino,978019727653199872,RT : Just like they’ve smeared protesters duri...
3,DLPBurke,978029941597130753,We have no laws here now. We gave up law and ...
4,DLPBurke,978029941597130753,We have no laws here now. We gave up law and ...
5,GettBoba,978031781806043136,Oxford my star; Muslim middle eastern rapists
6,GettBoba,978031781806043136,Oxford my star; Muslim middle eastern rapists
7,matantemorin,978071791292813312,RT : Notre qualité de vie et notre capacité de...
8,matantemorin,978071791292813312,RT : Notre qualité de vie et notre capacité de...
9,3thman_kh,978074226472181761,اللهم لا سهل إلا ما جعلته سهلاً وأنت تجعل الحز...


### Retweets

In [8]:
# First see how many retweets we have
curr.execute('SELECT count("retweet"), COUNT(*) - COUNT("retweet") from raw_tweets;')
num_valid_rt, num_null_rt = curr.fetchone()

print ('Retweets:\t{}\nOriginal Tweets\t{}'.format(num_valid_rt, num_null_rt))

Retweets:	181478
Original Tweets	436034


In [9]:
# Take a look at some retweets
rt_query = 'SELECT retweet FROM raw_tweets WHERE retweet IS NOT NULL LIMIT 100;'
rt_df = psql.read_sql(rt_query, conn)

rt_df.head(20)

,retweet
0,RT
1,RT
2,RT
3,RT
4,RT
5,RT
6,RT
7,RT
8,RT
9,RT


In [10]:
filter_retweets = 'SELECT COUNT(*) FROM raw_tweets WHERE retweet'

### Filter all

```
SELECT
    # Only select unduplicated tweets from database
    DISTINCT ON ("tweetID")
    *
FROM 
    raw_tweets
WHERE
    # Tweets need a message for consideration
    (message IS NOT NULL) AND
    # Don't grab retweets
    (retweet IS NULL) AND
    # Only grab english flagged tweets
    (LEFT(language, 2) LIKE 'en') AND
    # Only grab location verified tweets
    (latitude IS NOT NULL) AND
    (longitude IS NOT NULL);
```

In [19]:
filter_sql = """
SELECT
    DISTINCT ON ("tweetID")
    *
FROM 
    raw_tweets
WHERE
    (message IS NOT NULL) AND
    (retweet IS NULL) AND
    (LEFT(language, 2) LIKE 'en') AND
    (latitude IS NOT NULL) AND
    (longitude IS NOT NULL);
"""

filter_df = psql.read_sql(filter_sql, conn)

In [20]:
print ('Total:\t{}\nFiltered:\t{}'.format(num_total_tweets, len(filter_df)))
filter_df.head()

Total:	617512
Filtered:	119737


,id,tweetID,date,message,username,userID,language,longitude,latitude,retweet
0,346075,812080849239810048,2016-12-22 17:42:16,PrisonPlanetTV: RT PrisonPlanet: Muslim YouTub...,RJazz117,19497260,en,114.627525,-3.351165,None
1,346775,812081118619041792,2016-12-22 17:43:21,اللهم لا سهل إلا ما جعلته سهلاً وأنت تجعل الحز...,ghuwainm,2252611996,en,114.627525,-3.351165,None
2,346975,812081201355878406,2016-12-22 17:43:40,what religion were the flames ? Muslim I bet eh,__ZAINRAJA,48112908,en,114.627525,-3.351165,None
3,347503,812081414128615424,2016-12-22 17:44:31,Barack takes the utmost care to avoid offendi...,Asher_P_Fly,714798361,en,107.607830,-6.914070,None
4,347603,812081456121987072,2016-12-22 17:44:41,i was so proud of how far the UK had come in t...,TellDramaUK,3068652335,en,107.607830,-6.914070,None


### Select and insert into new table

In [23]:
sel_insert_sql = """INSERT INTO filter_tweets """ + filter_sql

In [24]:
sel_insert_sql

'INSERT INTO filter_tweets \nSELECT\n    DISTINCT ON ("tweetID")\n    *\nFROM \n    raw_tweets\nWHERE\n    (message IS NOT NULL) AND\n    (LEFT(language, 2) LIKE \'en\') AND\n    (latitude IS NOT NULL) AND\n    (longitude IS NOT NULL) AND\n    (retweet IS NULL);\n'

In [25]:
conn.close()